# 🎬 Movie Recommendation System
This notebook allows you to get movie recommendations based on a movie you like.

In [ ]:
# Importing dependencies
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load and preprocess dataset
DATA_PATH = "data/movies.csv"  # Make sure your CSV is here

def load_and_preprocess(path=DATA_PATH):
    movies_data = pd.read_csv(path)
    selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']

    # Replace null values with empty strings
    for feature in selected_features:
        if feature not in movies_data.columns:
            movies_data[feature] = ''
        movies_data[feature] = movies_data[feature].fillna('')

    # Combine selected features into a single string
    combined_features = (
        movies_data['genres'].astype(str) + ' ' +
        movies_data['keywords'].astype(str) + ' ' +
        movies_data['tagline'].astype(str) + ' ' +
        movies_data['cast'].astype(str) + ' ' +
        movies_data['director'].astype(str)
    )

    return movies_data, combined_features

In [ ]:
# Build similarity matrix
def build_similarity_matrix(combined_features):
    vectorizer = TfidfVectorizer()
    feature_vectors = vectorizer.fit_transform(combined_features)
    similarity = cosine_similarity(feature_vectors)
    return similarity

In [ ]:
# Recommend movies
def recommend_movies(movies_data, similarity, movie_name, top_n=10):
    list_of_all_titles = movies_data['title'].astype(str).tolist()
    find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

    if not find_close_match:
        print(f"❌ No match found for '{movie_name}'. Please check the spelling.")
        return []

    close_match = find_close_match[0]
    index_of_the_movie = movies_data[movies_data.title == close_match].index[0]

    similarity_score = list(enumerate(similarity[index_of_the_movie]))
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

    recommended = []
    for movie in sorted_similar_movies[1:top_n+1]:
        index = movie[0]
        title_from_index = movies_data.iloc[index]['title']
        recommended.append(title_from_index)

    print(f"\n🎥 Movies suggested for you based on '{close_match}':\n")
    for i, title in enumerate(recommended, 1):
        print(f"{i}. {title}")

    return recommended

In [ ]:
# Run interactively
try:
    movies_data, combined_features = load_and_preprocess()
except FileNotFoundError:
    print("Could not find 'data/movies.csv'. Please place your dataset file at data/movies.csv and try again.")
    raise

similarity = build_similarity_matrix(combined_features)

# Example interactive usage
movie_name = input("Enter your favourite movie name: ")
recommend_movies(movies_data, similarity, movie_name, top_n=10)